In [1]:
from selenium import webdriver
import time
import tqdm
import glob as gb
import winsound

In [2]:
# generate Webdriver object - Chrome version 79.0.3945.130 (64 bit)
driver = webdriver.Chrome('driver/chromedriver.exe')
driver.get('https://nid.naver.com/nidlogin.login')

input('>>> press Enter after log in comleted : ')

## <<< Login manually at this step >>>

## M1. Crawling for all menus of a certain group

In [25]:
def click_expand(group_id_str):
    # (optional) click expand buttton
    expand_btn = driver.find_element_by_xpath(f'//*[@id="{group_id_str}btn"]')
    if expand_btn.get_attribute('class') == 'down-btn':
        expand_btn.click()
        driver.implicitly_wait(3)
        pass
    pass

def navigate_main(expand_flag=False, group_id_str=''):
    driver.get('https://cafe.naver.com/mpihb')  # //cafe.naver.com/MyCafeIntro.nhn?clubid=23064234
    driver.implicitly_wait(1)
    
    if expand_flag:
        #  (optional) click expand buttton
        click_expand(group_id_str)
    pass

group_id = "group324"
flag = True

navigate_main(expand_flag=flag, group_id_str = group_id)

iter_num = len( driver.find_elements_by_xpath(f'//*[@id="{group_id}"]/li/a') )
for menu_idx in range(iter_num):
    
    # refresh window
    navigate_main(expand_flag=flag, group_id_str = group_id)

    # select menu
    menu_list = driver.find_elements_by_xpath(f'//*[@id="{group_id}"]/li/a')
    menu_text = menu_list[menu_idx].text
    menu_list[menu_idx].click()
    driver.implicitly_wait(5)
    print(f'Let me crawl {menu_text} menu of {group_id}...')
    
    # only for re-activating code
    if menu_idx < 10:
        print(f'{menu_text} menu is already completed!!! skip that...')
        continue

    # view 50 items
    driver.switch_to.frame('cafe_main')
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('//*[@id="listSizeSelectDiv"]/a').click()
    driver.find_element_by_xpath('//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()
    time.sleep(1)

    # crawl files of 50 items in each page
    page_num = len(driver.find_elements_by_xpath('//*[@id="main-area"]/div[6]/a'))
    for page_idx in range(page_num):

        page_list = driver.find_elements_by_xpath('//*[@id="main-area"]/div[6]/a')
        page_list[page_idx].click()
        time.sleep(3)

        article_num = len(driver.find_elements_by_xpath('//*[@id="main-area"]/div[4]/table/tbody/tr'))
        print(f'Start crawling {page_idx+1} page, {article_num} items')
        for idx in tqdm.tqdm_notebook(range(article_num)):
            
#             # only for re-activating code
#             if idx < 12:
#                 continue
            
            article_list = driver.find_elements_by_xpath('//*[@id="main-area"]/div[4]/table/tbody/tr')
            time.sleep(2)
            tmp_article = article_list[idx].find_element_by_tag_name('td').find_elements_by_tag_name('div')[2].find_element_by_tag_name('a')
            tmp_article.click()
            time.sleep(1)

            try:
                driver.find_element_by_xpath('//a[@class="atch_view m-tcol-c"]').click()
                btn_num = len(driver.find_elements_by_xpath('//div[@class="download_opt"]/a[1]'))
                if btn_num > 1 : print(f'>>> Warning!!! no. of items : {btn_num} (in article {idx+1} of {page_idx+1} page)')
                for btn_idx in range(btn_num):
                    driver.find_elements_by_xpath('//div[@class="download_opt"]/a[1]')[btn_idx].click()
                    time.sleep(3)
            except Exception as e:
                print(f'>>> Error!!!!! (in article {idx+1} of {page_idx+1} page) :\n',e)
                winsound.Beep(2000, 1000)

            driver.execute_script("window.history.go(-1)")
            time.sleep(3)

    print('>>> crawling successfully finishied')
    winsound.Beep(2000, 1000)

Let me crawl EBS 올림포스 2(정회원) menu of group324...
EBS 올림포스 2(정회원) menu is already completed!!! skip that...
Let me crawl EBS 올림포스 1(정회원) menu of group324...
EBS 올림포스 1(정회원) menu is already completed!!! skip that...
Let me crawl 16.수특(영어)(정회원) menu of group324...
16.수특(영어)(정회원) menu is already completed!!! skip that...
Let me crawl 16.수특(영독)(정회원) menu of group324...
16.수특(영독)(정회원) menu is already completed!!! skip that...
Let me crawl 16.수특(영어)문제분리(정회원) menu of group324...
16.수특(영어)문제분리(정회원) menu is already completed!!! skip that...
Let me crawl 16.수특(독해)문제분리(정회원) menu of group324...
16.수특(독해)문제분리(정회원) menu is already completed!!! skip that...
Let me crawl 16.수완(유형편)(2016)(정회원) menu of group324...
16.수완(유형편)(2016)(정회원) menu is already completed!!! skip that...
Let me crawl 16.수완(실전편)(2016)(정회원) menu of group324...
16.수완(실전편)(2016)(정회원) menu is already completed!!! skip that...
Let me crawl 2015 수능완성(정회원) menu of group324...
2015 수능완성(정회원) menu is already completed!!! skip that...
Let me 

C:\Users\Samuel\Anaconda3\envs\homeET\lib\site-packages\ipykernel_launcher.py:59: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


>>> Warning!!! no. of items : 2 (in article 13 of 1 page)
>>> Error!!!!! (in article 14 of 1 page) :
 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class="atch_view m-tcol-c"]"}
  (Session info: chrome=79.0.3945.117)

>>> Warning!!! no. of items : 2 (in article 16 of 1 page)

>>> crawling successfully finishied


## M2. Crawling only one menu of a certain group

In [24]:
def click_expand(group_id_str):
    # (optional) click expand buttton
    expand_btn = driver.find_element_by_xpath(f'//*[@id="{group_id_str}btn"]')
    if expand_btn.get_attribute('class') == 'down-btn':
        expand_btn.click()
        driver.implicitly_wait(3)
        pass
    pass

def navigate_main(expand_flag=False, group_id_str=''):
    driver.get('https://cafe.naver.com/mpihb')  # //cafe.naver.com/MyCafeIntro.nhn?clubid=23064234
    driver.implicitly_wait(1)
    
    if expand_flag:
        #  (optional) click expand buttton
        click_expand(group_id_str)
    pass

group_id = "group324"
flag = True
menu2crawl_idx = 9

navigate_main(expand_flag=flag, group_id_str = group_id)

for menu_idx in [menu2crawl_idx]:
    
    # refresh window
    navigate_main(expand_flag=flag, group_id_str = group_id)

    # select menu
    menu_list = driver.find_elements_by_xpath(f'//*[@id="{group_id}"]/li/a')
    menu_text = menu_list[menu_idx].text
    menu_list[menu_idx].click()
    driver.implicitly_wait(5)
    print(f'Let me crawl {menu_text} menu of {group_id}...')
    
    if menu_idx < menu2crawl_idx:
        print(f'{menu_text} menu is already completed!!! skip that...')
        continue
    elif menu_idx > menu2crawl_idx:
        print(f'>>> crawling only {menu_text} menu successfully finishied')
        break

    # view 50 items
    driver.switch_to.frame('cafe_main')
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('//*[@id="listSizeSelectDiv"]/a').click()
    driver.find_element_by_xpath('//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()
    time.sleep(1)

    # crawl files of 50 items in each page
    page_num = len(driver.find_elements_by_xpath('//*[@id="main-area"]/div[6]/a'))
    for page_idx in range(page_num):

        page_list = driver.find_elements_by_xpath('//*[@id="main-area"]/div[6]/a')
        page_list[page_idx].click()
        time.sleep(3)

        article_num = len(driver.find_elements_by_xpath('//*[@id="main-area"]/div[4]/table/tbody/tr'))
        print(f'Start crawling {page_idx+1} page, {article_num} items')
        for idx in tqdm.tqdm_notebook(range(article_num)):
            
            # only for re-activating code
            if idx < 13:
                continue
            
            article_list = driver.find_elements_by_xpath('//*[@id="main-area"]/div[4]/table/tbody/tr')
            time.sleep(3)
            tmp_article = article_list[idx].find_element_by_tag_name('td').find_elements_by_tag_name('div')[2].find_element_by_tag_name('a')
            tmp_article.click()
            time.sleep(1)

            try:
                driver.find_element_by_xpath('//a[@class="atch_view m-tcol-c"]').click()
                btn_num = len(driver.find_elements_by_xpath('//div[@class="download_opt"]/a[1]'))
                if btn_num > 1 : print(f'>>> Warning!!! no. of items : {btn_num} (in article {idx+1} of {page_idx+1} page)')
                for btn_idx in range(btn_num):
                    driver.find_elements_by_xpath('//div[@class="download_opt"]/a[1]')[btn_idx].click()
                    time.sleep(3)
            except Exception as e:
                print(f'>>> Error!!!!! (in article {idx+1} of {page_idx+1} page) :\n',e)
                winsound.Beep(2000, 1000)

            driver.execute_script("window.history.go(-1)")
            time.sleep(3)

    print('>>> crawling successfully finishied')
    winsound.Beep(2000, 1000)

Let me crawl 고1 모의고사(정회원) menu of group324...
Start crawling 1 page, 18 items


C:\Users\Samuel\Anaconda3\envs\homeET\lib\site-packages\ipykernel_launcher.py:61: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


>>> Warning!!! no. of items : 2 (in article 18 of 1 page)

>>> crawling successfully finishied
